# Dendogram Based Method for Clusteredness

## Info
### Resources
- [Wards Method](https://www.wikiwand.com/en/Ward%27s_method)
- [Scipy `linkage` documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html)

### Introduction
We made the following observation. When looking at dendogramms obtained via Wards method on gaussian mixture datasets, the number of undelying clusters could be visually inferred with surprising accuracy.
The process was as follows:
1. Have a look at how many branches are shown in the dendogramm for a given cut parallel to the x-axis
2. If this number stays constant for a "surprisingly" large range of y values, then this number is the number of clusters

The goal here is to formalize this observation and to make it applicable to any dataset.

### Method Description
Use the second discrete derivative of the distances of clusters after a merge of clusters as statistic.
Simulate the null distribution by calculating the statistic for standard normal or uniform distributed data.

### Possible Improvements and Extensions
- Adapt Wards method to the desired definition of "cluster" (i.e. Wards method => "clusteredness" means disconnected components)
- Use better null distributions, maybe different ones for each hypothesis of the form "there are k clusters"
- Combine p-values for different k in a better way than Benferoni correction
- Use a better statistic than the second discrete derivative

### Unexplained Observations
- The statistic seems to have some structure that could be exploited, specifically each value seams to be anticorrelated with the next one
- In 32d with 7 clusters the statistic seems to find some hirarchy, which is not always present in the first 2 principal components (with hirarchy I mean that the statistic seems to confirm that also clusterings with fewer clusters are valid)

## Method

In [ ]:
import functools

import numpy as np
import matplotlib.pyplot as plt

import scipy.cluster.hierarchy as scihi


def discrete_diff2(x: np.ndarray) -> np.ndarray:
    """
    Compute the second discrete derivative of a vector x.

    Parameters
    ---
    x: np.ndarray
        The vector to compute the second derivative of.

    Returns
    ---
    np.ndarray
        The second derivative of x.
    """
    if len(x) < 2:
        raise ValueError("x must have at least two elements")

    if x.ndim != 1:
        raise ValueError("x must be a vector")

    x_prime = np.diff(x)
    x_prime_prime = np.diff(x_prime)
    return x_prime_prime


def clusteredness_statistics(data: np.ndarray, cutoff: int) -> np.ndarray:
    # normalize data for comparability to null model
    data = data - data.mean(axis=0, keepdims=True)
    data = data / data.std(axis=0, keepdims=True)

    linkage_matrix = scihi.linkage(data, method="ward")

    distances = linkage_matrix[:, 2]
    distance_diff2 = discrete_diff2(distances[::-1])
    distance_diff2 = distance_diff2[:cutoff]

    return distance_diff2


@functools.lru_cache(maxsize=128)
def clusteredness_null_distribution(
    dim: int,
    num_samples: int,
    cutoff: int,
    iterations: int,
    disttype: str = "normal",
) -> np.ndarray:
    statistics_list = []

    for _ in range(iterations):
        if disttype == "normal":
            data = np.random.normal(size=(num_samples, dim))
        
        elif disttype == "uniform":
            data = np.random.uniform(size=(num_samples, dim))

        else:
            raise ValueError(f"Unknown distribution type {disttype}")
        
        statistics = clusteredness_statistics(data, cutoff=cutoff)
        statistics_list.append(statistics)

    statistics = np.stack(statistics_list, axis=0)
    statistics = np.sort(statistics, axis=0)
    return statistics


def clusteredness_pvalues(
    data: np.ndarray,
    cutoff: int,
    iterations: int = 10000,
):
    num_samples, dim = data.shape

    null_distribution = clusteredness_null_distribution(
        dim=dim,
        num_samples=num_samples,
        cutoff=cutoff,
        iterations=iterations,
    )

    pvalues = []
    data_stats = clusteredness_statistics(data, cutoff=cutoff)

    for null_stat, data_stat in zip(null_distribution.T, data_stats):
        pvalue = 1 - np.searchsorted(null_stat, data_stat) / iterations
        pvalues.append(pvalue)

    return np.stack(pvalues, axis=0)

### Null Distribution

In [ ]:
cutoff = 8
null_dist = clusteredness_null_distribution(
    2,
    256,
    cutoff=cutoff,
    iterations=10000,
)

fig, ax = plt.subplots(figsize=(6, 4))
ax.boxplot(null_dist, showfliers=False)

ax.set_xticks(
    np.arange(1, cutoff + 1),
    np.arange(1, cutoff + 1) + 1,
)

ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Distance of Cluster Merge")

ax.set_title("Null Distribution of Clusteredness Statistics for Normal Data")
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))

cmap = plt.get_cmap("turbo")

for i in range(1, cutoff + 1):
    ax.hist(
        null_dist[:, i - 1],
        bins=32,
        label=f"{i} Clusters",
        histtype="step",
        color=cmap(i / cutoff),
        linewidth=2,
    )

ax.set_xlabel("Distance of Cluster Merge")
ax.set_ylabel("Frequency")
ax.set_title("Null Distribution of Clusteredness Statistics for Normal Data")
ax.legend(frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
null_dist_uniform = clusteredness_null_distribution(
    2,
    256,
    cutoff=cutoff,
    iterations=10000,
    disttype="uniform",
)

fig, ax = plt.subplots(figsize=(6, 4))
ax.boxplot(null_dist_uniform, showfliers=False)

ax.set_xticks(
    np.arange(1, cutoff + 1),
    np.arange(1, cutoff + 1) + 1,
)

ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Distance of Cluster Merge")

ax.set_title("Null Distribution of Clusteredness Statistics for Uniform Data")
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))

cmap = plt.get_cmap("turbo")

for i in range(1, cutoff + 1):
    ax.hist(
        null_dist_uniform[:, i - 1],
        bins=32,
        label=f"{i} Clusters",
        histtype="step",
        color=cmap(i / cutoff),
        linewidth=2,
    )

ax.set_xlabel("Distance of Cluster Merge")
ax.set_ylabel("Frequency")
ax.set_title("Null Distribution of Clusteredness Statistics for Uniform Data")
ax.legend(frameon=False)

plt.tight_layout()
plt.show()

## On Data

In [ ]:
import sys

sys.path.append("../../src/")

from corc.generation import GenerationModel

### Equidistant Triangle

In [ ]:
params = {
    "center_structure": "equidistant_triangle",
    "n_centers": 3,
    "distance": 1,
    "n_samples": 1000,
    "dim": 2,
    "save_file": False,
    "outdir": ".",
}

gen = GenerationModel(**params)
gen.generate()

cutoff = 8
nstds = 5

for i, std in enumerate(np.logspace(-2, 0, nstds)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    for i in range(32):
        data = gen.sample_embedding(std=std)

        if i == 0:
            ax1.scatter(data[:, 0], data[:, 1], c=gen.labels, cmap="tab10")
            ax1.set_xlabel("$x_1$")
            ax1.set_ylabel("$x_2$")

        pvalues = clusteredness_pvalues(data, cutoff=cutoff, iterations=1000)
        ax2.plot(
            np.arange(1, cutoff + 1),
            pvalues,
            label=f"std={std:.2f}",
            linewidth=2,
            color="tab:blue",
            alpha=0.5,
        )

    ax2.set_xticks(
        np.arange(1, cutoff + 1),
        np.arange(1, cutoff + 1) + 1,
    )

    ax2.set_xlabel("Number of Clusters")
    ax2.set_ylabel("P-value of Cluster Merge")
    fig.suptitle(f"Equidistant Triangle with $\\sigma$={std:.2f}")

### More Clusters

In [ ]:
params = {
    "center_structure": "uniform",
    "n_centers": 7,
    "distance": 1,
    "n_samples": 1000,
    "dim": 2,
    "save_file": False,
    "outdir": ".",
}

gen = GenerationModel(**params)
gen.generate()

cutoff = 8
nstds = 5

for i, std in enumerate(np.logspace(-2, 0, nstds)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    for i in range(32):
        data = gen.sample_embedding(std=std)

        if i == 0:
            ax1.scatter(data[:, 0], data[:, 1], c=gen.labels, cmap="tab10")
            ax1.set_xlabel("$x_1$")
            ax1.set_ylabel("$x_2$")

        pvalues = clusteredness_pvalues(data, cutoff=cutoff, iterations=1000)
        ax2.plot(
            np.arange(1, cutoff + 1),
            pvalues,
            label=f"std={std:.2f}",
            linewidth=2,
            color="tab:blue",
            alpha=0.5,
        )

    ax2.set_xticks(
        np.arange(1, cutoff + 1),
        np.arange(1, cutoff + 1) + 1,
    )

    ax2.set_xlabel("Number of Clusters")
    ax2.set_ylabel("P-value of Cluster Merge")
    fig.suptitle(f"7 Clusters with Uniform Centers and $\\sigma$={std:.2f}")

### More Dimensions

In [ ]:
from sklearn.decomposition import PCA

params = {
    "center_structure": "uniform",
    "n_centers": 3,
    "distance": 1,
    "n_samples": 1000,
    "dim": 32,
    "save_file": False,
    "outdir": ".",
}

gen = GenerationModel(**params)
gen.generate()

cutoff = 8
nstds = 5

for i, std in enumerate(np.logspace(-2, 0, nstds)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    for i in range(32):
        data = gen.sample_embedding(std=std)

        if i == 0:
            # project data to 2D
            pca = PCA(n_components=2)
            data_pca = pca.fit_transform(data)

            ax1.scatter(
                data_pca[:, 0],
                data_pca[:, 1],
                c=gen.labels,
                cmap="tab10",
            )
            ax1.set_xlabel("first principal component")
            ax1.set_ylabel("second principal component")

        pvalues = clusteredness_pvalues(data, cutoff=cutoff, iterations=1000)
        ax2.plot(
            np.arange(1, cutoff + 1),
            pvalues,
            label=f"std={std:.2f}",
            linewidth=2,
            color="tab:blue",
            alpha=0.5,
        )

    ax2.set_xticks(
        np.arange(1, cutoff + 1),
        np.arange(1, cutoff + 1) + 1,
    )

    ax2.set_xlabel("Number of Clusters")
    ax2.set_ylabel("P-value of Cluster Merge")
    fig.suptitle(f"32 Dimensional & 3 Clusters with $\\sigma$={std:.2f}")

In [ ]:
from sklearn.decomposition import PCA

params = {
    "center_structure": "uniform",
    "n_centers": 7,
    "distance": 1,
    "n_samples": 1000,
    "dim": 32,
    "save_file": False,
    "outdir": ".",
}

gen = GenerationModel(**params)
gen.generate()

cutoff = 8
nstds = 5

for i, std in enumerate(np.logspace(-2, 0, nstds)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    for i in range(32):
        data = gen.sample_embedding(std=std)

        if i == 0:
            # project data to 2D
            pca = PCA(n_components=2)
            data_pca = pca.fit_transform(data)

            ax1.scatter(
                data_pca[:, 0],
                data_pca[:, 1],
                c=gen.labels,
                cmap="tab10",
            )
            ax1.set_xlabel("first principal component")
            ax1.set_ylabel("second principal component")

        pvalues = clusteredness_pvalues(data, cutoff=cutoff, iterations=1000)
        ax2.plot(
            np.arange(1, cutoff + 1),
            pvalues,
            label=f"std={std:.2f}",
            linewidth=2,
            color="tab:blue",
            alpha=0.5,
        )

    ax2.set_xticks(
        np.arange(1, cutoff + 1),
        np.arange(1, cutoff + 1) + 1,
    )

    ax2.set_xlabel("Number of Clusters")
    ax2.set_ylabel("Distance of Cluster Merge")
    fig.suptitle(f"32 Dimensional & 7 Clusters with $\\sigma$={std:.2f}")